<a href="https://colab.research.google.com/github/St0rmMaster/jess/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22German_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot
!pip install aiogram
!pip install nest_asyncio

In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, ChatPermissions
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes, MessageHandler, filters, Updater
import nest_asyncio
nest_asyncio.apply()
import sqlite3
import logging

In [ ]:
def connect_db():
    conn = sqlite3.connect('db.sqlite')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS attempted_users (
        user_id INTEGER PRIMARY KEY,
        username NVARCHAR(255),
        question_number INTEGER  -- Добавляем колонку question_number
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS bot_settings (
        setting_key NVARCHAR(255) PRIMARY KEY,
        setting_value TEXT
    )''')
    conn.commit()
    return conn

In [ ]:
TOKEN = '7011070020:AAGS3jUQhPPmeUrJQX2AdgnVNMI2ME2339M'
CHANNEL_ID = '-1001993399856'
ADMIN_CHAT_ID = '-1002097755337'
COMMENT_CHAT_ID = '-1002101294439'

In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_id = update.effective_user.id
    conn = connect_db()
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT 1 FROM attempted_users WHERE user_id = ?", (user_id,))
        already_attempted = cursor.fetchone()
        if already_attempted:
            await update.message.reply_text("Вы уже проходили этот тест. Повторное прохождение не разрешено.Обратитесь за помощью к @CaptainGERMAN")
            return
        else:
            cursor.execute("INSERT INTO attempted_users (user_id, username, question_number) VALUES (?, ?, ?)", (user_id, update.effective_user.username, 1))
            conn.commit()
            await ask_question(context, user_id, 1, None)
    finally:
        conn.close()

In [ ]:
async def ask_question(context, user_id, question_number, message):
    questions = [
        "В 2014 году Россия произвела аннексию полуострова Крым, тем самым нарушив все нормы международного права. Крым должен быть возвращён в состав Украины безоговорочно.",
        "В феврале 2022 года Россия преступно и предательски напала на Украину. Развязав кровопролитную, захватническую войну.",
        "Путин - международный преступник, предводитель захватившей власть мафиозной группировки. Убийца и лжец. Его место в тюрьме.",
        "Я согласен с тем, что оскорбления других участников чата, а также любая дискриминация по расовому признаку недопустимы. За нарушение данного пункта следует перманентный бан."
    ]
    keyboard = [
        [InlineKeyboardButton("Согласен", callback_data=f"agree_{question_number}"),
         InlineKeyboardButton("Не согласен", callback_data=f"disagree_{question_number}")],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    if message is None or 'edit_text' not in dir(message):
        await context.bot.send_message(chat_id=user_id, text=questions[question_number - 1], reply_markup=reply_markup)
    else:
        await message.edit_text(text=questions[question_number - 1], reply_markup=reply_markup)

In [ ]:
async def handle_response(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    query = update.callback_query
    await query.answer()
    user_id = query.from_user.id
    data = query.data
    conn = connect_db()
    cursor = conn.cursor()
    try:
        response, question_number = data.split('_')
        question_number = int(question_number)
        if question_number >= 4 or response == "disagree":
            if response == "disagree":
                await query.edit_message_text(text="Русский военный корабль... ну ты понял. Вате тут не рады.", reply_markup=None)
                await context.bot.ban_chat_member(chat_id=CHANNEL_ID, user_id=user_id)
                await context.bot.ban_chat_member(chat_id=COMMENT_CHAT_ID, user_id=user_id)
                await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text=f"Пользователь {query.from_user.full_name} (ID: {user_id}) не прошел тест и был заблокирован.")
            else:
                member = await context.bot.get_chat_member(chat_id=CHANNEL_ID, user_id=user_id)
                if member.status in ['left', 'kicked']:
                    invite_link = await context.bot.create_chat_invite_link(chat_id=CHANNEL_ID, member_limit=1)
                    await context.bot.restrict_chat_member(chat_id=COMMENT_CHAT_ID, user_id=user_id, permissions=ChatPermissions(can_send_messages=True))
                    await context.bot.send_message(chat_id=user_id, text=f"Мы рады свободным, адекватным и честным людям. Добро пожаловать в сообщество! Вот ваша ссылка для вступления в канал: {invite_link.invite_link}")
                    await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text=f"Пользователь {query.from_user.full_name} (ID: {user_id}) успешно прошел тест и стал подписчиком канала.")
                else:
                    await context.bot.restrict_chat_member(chat_id=COMMENT_CHAT_ID, user_id=user_id, permissions=ChatPermissions(can_send_messages=True))
                    await query.edit_message_text(text="Мы рады свободным, адекватным и честным людям. Теперь вы можете оставлять комментарии.", reply_markup=None)
                    await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text=f"Подписчик канала {query.from_user.full_name} (ID: {user_id}) успешно прошел тест и получил доступ к комментированию.")
        else:
            new_question_number = question_number + 1
            cursor.execute("UPDATE attempted_users SET question_number = ? WHERE user_id = ?", (new_question_number, user_id))
            await ask_question(context, user_id, new_question_number, query.message)
        conn.commit()
    except Exception as e:
        await query.edit_message_text(text="Произошла ошибка. Это не отрицательный результат теста. Пожалуйста, сделайте скрин этого сообщения и обратитесь за помощью к @CaptainGERMAN.")
    finally:
        conn.close()

In [ ]:
async def is_user_admin(context: ContextTypes.DEFAULT_TYPE, user_id: int) -> bool:
    try:
        member = await context.bot.get_chat_member(ADMIN_CHAT_ID, user_id)
        return member.status in ['administrator', 'creator']
    except Exception as e:
        logger.error(f"Ошибка при проверке статуса администратора: {e}")
        return False

In [ ]:
async def text_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    if 'setting_to_change' in context.user_data:
        new_value = update.message.text
        setting_key = context.user_data['setting_to_change']
        user_id = update.effective_user.id

        # Проверяем, что сообщение отправлено в административный чат администратором
        if str(update.message.chat_id) == ADMIN_CHAT_ID and await is_user_admin(context, user_id):
            # Обновляем настройку в базе данных
            conn = connect_db()
            cursor = conn.cursor()
            cursor.execute("UPDATE bot_settings SET setting_value = ? WHERE setting_key = ?", (new_value, setting_key))
            conn.commit()
            conn.close()

            await update.message.reply_text(f"Настройка '{setting_key}' обновлена на '{new_value}'.")
            del context.user_data['setting_to_change']  # Очищаем выбранную настройку
        else:
            await update.message.reply_text('У вас нет прав для выполнения этой операции.')

In [ ]:
async def clear_all(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_id = update.effective_user.id
    if str(update.effective_chat.id) == ADMIN_CHAT_ID and await is_user_admin(context, user_id):
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute('DELETE FROM attempted_users')
        conn.commit()
        conn.close()
        await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text="Все данные успешно очищены.")
    else:
        await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text="У тебя нет прав для выполнения этой команды.")
async def clear_user(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_id = update.effective_user.id
    if str(update.effective_chat.id) == ADMIN_CHAT_ID and await is_user_admin(context, user_id):
        await update.message.reply_text("Пожалуйста, отправьте юзернейм пользователя для удаления")
        context.user_data['awaiting_username'] = True
    else:
        await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text="У тебя нет прав для выполнения этой команды.")

In [ ]:
async def main():
    application = Application.builder().token(TOKEN).build()
    application.add_handler(CommandHandler('start', start))
    application.add_handler(CommandHandler('clear_all', clear_all))
    application.add_handler(CommandHandler('clear_user', clear_user))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, text_handler))
    application.add_handler(CallbackQueryHandler(handle_response))
    await application.run_polling()

In [ ]:
nest_asyncio.apply()

await main()